# Korpusverarbeitung – Annotation mit spaCy

## Übersicht
Im Folgenden wird exemplarisch ein Text (txt-Datei) mit der Bibliothek [spaCy](https://spacy.io) annotiert. Dafür werden folgendene Schritte durchgeführt:
1. Einlesen des Texts
2. Worthäufigkeiten ohne echte Tokenisierung
   * Aufteilen des Texts in Wörter auf Grundlage von Leerzeichen
   * Abfrage von Häufigkeiten
4. Annotation mit spaCy
   * Laden des Sprachmodells
   * Analysekomponenten auswählen
   * Text annotieren
   * Worthäufigkeiten anzeigen
5. Annotation speichern
6. Prozess für das gesamte Korpus ausführen

In [ ]:
# Laden von Bibliotheken 
from pathlib import Path
from time import time
from collections import OrderedDict, Counter

from tqdm import tqdm
import pandas as pd
import numpy as np
import spacy

import panel as pn
pn.extension()
import bokeh

## 1. Einlesen des Texts
Um eine Datei mit Python bearbeiten zu können, muss die Datei zuerst ausgewählt werden, d.h der [Pfad](https://en.wikipedia.org/wiki/Path_(computing)) zur Datei wird gesetzt und dann eingelesen werden. 

In [ ]:
text_path = Path("../data/txt/SNP2719372X-19181015-0-0-0-0.txt")
text = text_path.read_text()
print(f"Textauszug:\n {text[10280:10400]}")

## 2. Worthäufigkeiten ohne echte Tokenisierung

### 2.1 Text in Wörter aufteilen
Der einfachste Weg einen Text automatisch in Wörter aufzuteilen, ist anzunehmen, das Wörter durch Leerzeichen getrennt sind. 

In [ ]:
words = text.split()

**Prüfen**: Wie sieht die Wortliste aus?

In [ ]:
words[100:120]

Wie viele Wörter gibt es insgesamt?

In [ ]:
len(words)

Wie zu sehen ist, hat diese Art der "falschen" Tokenisierung den Nachteil, dass Satzzeichen nicht von Wörtern abgetrennt werden. \
Die Wortanzahl ist dementsprechend auch nicht akkurat. 

### 2.2 Anzeigen von Worthäufigkeiten
Auf Grundlage dieser Wortliste kann trotzdem schon eine erste basale Häufigkeitenabfrage erfolgen: 

In [ ]:
word_frequencies = Counter(words)

In [ ]:
def lookup_word_frequency(word):
    return word_frequencies.get(word, 0)

word_input = pn.widgets.TextInput(name='Wort', placeholder='Geben Sie ein Wort ein:')

@pn.depends(word_input)
def update_output(word):
    frequency = lookup_word_frequency(word)
    return pn.pane.Markdown(f"#### Die Häufigkeit des Worts **'{word}'** ist: {frequency}")
pn.Column(word_input, update_output).servable()

## 3. Annotation mit spaCy
Um eine präzisere Einteilung in Wörter zu erhalten (Tokenisierung) und um flektierte Wörter aufeinander abbildbar zu machen (Lemmatisierung), wird der Text im folgenden durch die Bibliothek [spaCy](https://spacy.io/) annotiert. Dafür werden folgende Schritte ausgeführt:
1. Das sprachspezifische Modell wird geladen. Wir arbeiten mit dem weniger akkuraten aber schnellsten spaCy Modell `de_core_news_sm`. 
2. Für eine erhöhte Annotationsgeschwindigkeit werden nur bestimmte Analysekomponenten geladen. Dies ist vor allem für größere Textmengen sinnvoll.
3. Der Text wird annotiert und die Token sowie die dazugehörigen Lemmata werden extrahiert.

### 3.1 Sprachmodell laden
Das sprachspezifische Modell wird geladen. Es handelt sich dabei um das am wenigsten akkurate aber schnellste Modell. 

In [ ]:
nlp = spacy.load('de_core_news_sm')

### 3.2 Analysekomponenten auswählen
Es werden einige Analysekomponent wie z. B. das Aufteilen des Texts in Sätze (sentencizer) oder die [Named Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition) (ner) ausgeschlossen, da diese für die Tokenisierung und die Lemmatisierung nicht benötigt werden. Der Auschluss der Komponentnen erhöht die Annotationsgeschwindikgeit. 

In [ ]:
disable_components = ['ner', 'morphologizer', 'attribute_ruler', 'sentencizer']

### 3.3 Annotieren der Texte: Token, Lemma
Der ausgewählte Text wird mit spaCy annotiert und die Token sowie die dazugehörigen Lemmata werden extrahiert und in einer Tabelle gespeichert. Das Tabellenformat wurde gewählt, da sich darin gut relationale Daten speichern lassen.

In [ ]:
current = time()
doc = nlp(text)
text_annotated = {}
text_annotated['Token'] = [tok.text for tok in doc]
text_annotated['Lemma'] = [tok.lemma_ for tok in doc]
text_annotated_df = pd.DataFrame(text_annotated)
took = time() - current
print(f"Die Annotation hat {round(took, 2)} Sekunden gedauert.") 

Auszug aus der Tabelle, in der der annotierte Text gespeichert ist:

In [ ]:
text_annotated_df.head()

### 3.4 Worthäufigkeit mit echter Tokenization   

Durch die Tokenisierung wurden z. B. Satzzeichen von Wörtern abgetrennt. An der Textlänge lässt sich dies schon erkennen. 

In [ ]:
text_tokenized = text_annotated_df.Lemma
len(text_tokenized)

Auf Grundlage des tokenisierten und lemmatisierten Texts, kann die Häufigkeitenabfrage erneut augeführt werden. Da durch die Lemmatisierung flektierte Wortformen auf die Grundformen zurückgeführt wurden, erwarten wir, dass die Häufigkeit einer Wortgrundform im Gegensatz zur vorherigen Abfrage erhöht ist. 

In [ ]:
token_frequencies = Counter(text_tokenized)


In [ ]:
def lookup_token_frequency(token):
    return token_frequencies.get(token, 0)

token_input = pn.widgets.TextInput(name='Token', placeholder='Geben Sie ein Wort ein:')

@pn.depends(token_input)
def update_output_token(token):
    frequency = lookup_token_frequency(token)
    return pn.pane.Markdown(f"#### Die Häufigkeit des Token **'{token}'** ist: {frequency}")
pn.Column(token_input, update_output_token).servable()

## 4. Annotation speichern
Um den annotierten Text zu speichern, wird zuerst der Dateiname festgelegt. Dafür wird die Dateiendung ersetzt von `.txt` zu `.csv`.

[CSV](https://en.wikipedia.org/wiki/Comma-separated_values) (comma-separated value) ist das Standardformat um tabellarische Daten im Klartext zu speichern. 

In [ ]:
output_path = Path(r"../data/csv") / text_path.with_suffix(".csv").name
text_annotated_df.to_csv(output_path, index=False)

## 5. Prozess für das gesamte Korpus ausführen 

In [ ]:
def read_corpus_linewise(corpus_dir: Path) -> OrderedDict[str,str]:
    """
    Reads txt files from a given directory. Returns a dictionary with the filename
    as key and the txt file content as value.
    :param Path corpus_dir: The directory in which the txt files are saved
    :return OrderedDict[str, str]: The file names as keys, the file content as value
    """
    corpus = OrderedDict()
    for filepath in corpus_dir.iterdir():
        if filepath.suffix == ".txt":
            text = filepath.read_text()
            corpus[filepath.name] = text
    return corpus

def annotate_corpus(corpus: OrderedDict[str, str], disable_components: list[str]) -> dict[str, pd.DataFrame]:
    """
    Annotate a corpus (filename: text) with spacy. Collect the Token and Lemma information. 
    Save the annotation information as a pandas DataFrame. 
    :param OrderedDict[str, str] corpus: The file names as keys, the file content as value
    :param list[str] disable_components: spacy components to be diasbled in the annotation process
    :return dict[str, pd.DataFrame]: The file name as keys, the annotated text as value
    """
    # list to collect how long the annotation runs take in seconds
    took_per_text = []

    # define result dict
    corpus_annotated = {}
    
    filename_list = list(corpus.keys())
    current = time()
    
    # iterate over the corpus values, annotate them with spacy
    for i, doc in tqdm(enumerate(nlp.pipe(list(corpus.values()), disable=disable_components))):
        before = current
        current = time()
        took_per_text.append(current - before)

        # Save the token and lemma information to a dictionary
        text_annotated = {}
        text_annotated['Token'] = [tok.text for tok in doc]
        text_annotated['Lemma'] = [tok.lemma_ for tok in doc]

        # Save the annotation as pandas DataFrame to the result dict
        # Key is the current filename
        corpus_annotated[filename_list[i]] = pd.DataFrame(text_annotated)

    # print corpus size and performance
    print(f"""Processed {len(corpus_annotated)} texts with spacy.
    Took {round(np.mean(took_per_text), 4)} seconds per text on average.
    Took {round(np.sum(took_per_text) / 60, 4)} minutes in total.""")

    return corpus_annotated

In [ ]:
# Korpus einlesen
corpus_dir = Path(r"../data/txt/")
corpus = read_corpus_linewise(corpus_dir)
print(f"Es wurden {len(corpus)} Dateien eingelesen.")

# Korpus annotieren
corpus_annotated = annotate_corpus(corpus, disable_components)

# Annotiertes Korpus speichern
output_dir = Path(r"../data/csv")
for filepath, text_annotated in corpus_annotated.items():
    filepath = Path(filepath)
    output_path = output_dir / filepath.with_suffix(".csv")
    text_annotated.to_csv(output_path, index=False)